def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [1]:
import pandas as pd
import ast

In [2]:
steam_games = pd.read_csv("..\\Datsets\\CSV\\steam_games.csv")
user_items =pd.read_csv("..\\Datsets\\CSV\\user_items.csv")

In [3]:
# Unir los DataFrames usando las columnas 'id' en steam_games y 'item_id' en user_items
df_UserForGenre = pd.merge(user_items, steam_games, left_on='item_id', right_on='id', how='inner')
# Seleccionar las columnas requeridas
selected_columns = ['genres', 'release_year', 'playtime_forever', 'user_id']
# Crear un nuevo DataFrame con las columnas seleccionadas
df_UserForGenre = df_UserForGenre[selected_columns]
#Convertimos la columna genres a tipo lista
# Verificar si 'genres' contiene NaN antes de aplicar ast.literal_eval
df_UserForGenre['genres'] = df_UserForGenre['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
#Expandimos las listas en columnas
df_UserForGenre = df_UserForGenre.explode('genres').reset_index(drop=True)
# Agrupar por 'genres', 'release_year' y 'user_id' y sumar 'playtime_forever'
df_UserForGenre = df_UserForGenre.groupby(['genres', 'release_year', 'user_id'], as_index=False)['playtime_forever'].sum()



In [4]:
# Agrupar por 'genres', 'user_id' y sumar 'release_year'
grouped_df = df_UserForGenre.groupby(['genres', 'user_id'], as_index=False)['release_year'].sum()

# Encontrar el 'user_id' para cada género donde la suma de 'release_year' es máxima
idx_max_years_sum = grouped_df.groupby('genres')['release_year'].idxmax()
result_df = grouped_df.loc[idx_max_years_sum]

# Filtrar df_UserForGenre para mantener solo las filas con un único usuario por cada género
df_UserForGenre  = df_UserForGenre[df_UserForGenre.set_index(['genres', 'user_id']).index.isin(result_df.set_index(['genres', 'user_id']).index)]


In [6]:
#Convertimos a CSV
df_UserForGenre.to_csv('..\\Datsets\\archivos_API\\UserForGenre', index=False)

In [22]:
df_UserForGenre.head(10)

,genres,release_year,user_id,playtime_forever
73,Action,1983.0,djnemonic,103
253,Action,1988.0,djnemonic,222
380,Action,1989.0,djnemonic,0
1063,Action,1990.0,djnemonic,0
1486,Action,1991.0,djnemonic,0
1673,Action,1992.0,djnemonic,0
2806,Action,1993.0,djnemonic,860
4891,Action,1994.0,djnemonic,271
8995,Action,1995.0,djnemonic,928
11852,Action,1996.0,djnemonic,669


In [27]:
def UserForGenre(genre):
    # Filtrar el DataFrame por el género dado
    genre_data = df_UserForGenre[df_UserForGenre['genres'] == genre]

    # Crear una lista de diccionarios para las horas jugadas por año
    hours_by_year = [{'Año': int(year), 'Horas': playtime} for year, playtime in genre_data[['release_year', 'playtime_forever']].values]

    # Crear el diccionario de salida
    result = {
        "Usuario con más horas jugadas para Género " + genre: genre_data.iloc[0]['user_id'],
        "Horas jugadas": hours_by_year
    }

    return result


In [29]:
genre_result = UserForGenre('Indie')
print(genre_result)

{'Usuario con más horas jugadas para Género Indie': 'chidvd', 'Horas jugadas': [{'Año': 1988, 'Horas': 0.0}, {'Año': 1995, 'Horas': 0.0}, {'Año': 1996, 'Horas': 0.0}, {'Año': 1997, 'Horas': 144.0}, {'Año': 1998, 'Horas': 75.0}, {'Año': 1999, 'Horas': 0.0}, {'Año': 2000, 'Horas': 247.0}, {'Año': 2001, 'Horas': 4.0}, {'Año': 2002, 'Horas': 7.0}, {'Año': 2003, 'Horas': 196.0}, {'Año': 2004, 'Horas': 25.0}, {'Año': 2005, 'Horas': 235.0}, {'Año': 2006, 'Horas': 1671.0}, {'Año': 2007, 'Horas': 102.0}, {'Año': 2008, 'Horas': 2449.0}, {'Año': 2009, 'Horas': 4379.0}, {'Año': 2010, 'Horas': 7888.0}, {'Año': 2011, 'Horas': 16597.0}, {'Año': 2012, 'Horas': 35970.0}, {'Año': 2013, 'Horas': 62205.0}, {'Año': 2014, 'Horas': 90429.0}, {'Año': 2015, 'Horas': 165180.0}, {'Año': 2016, 'Horas': 76531.0}, {'Año': 2017, 'Horas': 3604.0}, {'Año': 2018, 'Horas': 285.0}]}


In [28]:
df_UserForGenre['genres'].unique()

array(['Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing',
       'Simulation', 'Software Training', 'Sports', 'Strategy',
       'Utilities', 'Video Production', 'Web Publishing'], dtype=object)